In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid

In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

In [12]:
train_df = pd.read_csv('data/train.csv')

In [13]:
train_df.head()

,feature-0,feature-1,feature-2,feature-3,feature-4,feature-5,feature-6,feature-7,feature-8,feature-9,...,feature-1515,feature-1516,feature-1517,feature-1518,feature-1519,feature-1520,feature-1521,feature-1522,feature-1523,y
0,37.977273,6.758452,3.636364,10.792929,160.801682,151.109783,1.791689,6.818675,8.138413,8.270161,...,5.658393,4.151040,4.540632,4.953183,5.351562,5.311048,5.560922,5.643015,5.715999,0.0
1,19.408163,5.933978,2.816327,5.877551,162.949911,76.153796,1.381401,6.002651,5.080499,7.514421,...,4.830811,3.817712,4.123094,4.426343,4.823804,4.652173,4.795274,4.860781,5.001426,0.0
2,40.265306,7.425645,3.734694,13.160998,172.099640,161.790879,1.603976,7.410120,10.114794,8.805738,...,6.397659,4.223177,4.685597,5.116870,5.333926,5.504569,5.797956,6.009581,6.200889,0.0
3,43.976744,7.648293,3.837209,14.392765,168.885456,175.277251,1.622298,7.629033,12.180817,9.070719,...,5.879135,4.280132,4.563045,5.007714,5.159773,5.393628,5.640132,5.472271,5.741399,0.0
4,24.320988,6.534011,3.567901,8.913580,163.076959,96.019681,1.380679,6.566695,4.417010,8.058783,...,8.148663,4.624973,5.173321,5.720312,6.259342,6.626469,7.062406,7.472998,7.829842,0.0


In [14]:
train_df.y.value_counts()

1.0    790
0.0    315
Name: y, dtype: int64

In [59]:
# I have element-wise multiplication 
# Add trainable params
class WeightedInput(torch.nn.Module):
    def __init__(self):
        super(WeightedInput, self).__init__()
        # in_features = number of columns
        self.in_features = x.size()[1]
        self.weight = Parameter(torch.Tensor(1, self.in_features))
#         self.weight = Parameter(torch.tensor([0.1, 0.01, 0.001]))
    
    def forward(self, x):
        out = self.weight * x
        return out

In [58]:
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(3,3)
# y = torch.randn(N, D_in)

# Construct our model by instantiating the class defined above
model = WeightedInput()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
# criterion = torch.nn.MSELoss(reduction='sum')
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

    # Forward pass: Compute predicted y by passing x to the model
y_pred = model(x)
print(x)
print(y_pred)

tensor([[ 0.2215, -1.1257,  1.5885],
        [ 1.2855, -0.1729, -0.3832],
        [ 1.2759,  0.9707,  0.1470]])
tensor([[ 0.0221, -0.0113,  0.0016],
        [ 0.1285, -0.0017, -0.0004],
        [ 0.1276,  0.0097,  0.0001]], grad_fn=<MulBackward0>)


In [ ]:
class MyCustomLoss(Function):

    def forward(self, input, target):
        ... # implementation
        return loss   # a single number (averaged loss over batch samples)

    def backward(self, grad_output):
        ... # implementation
        return grad_input, None

In [ ]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [29]:
x = torch.tensor([1, 2])
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


In [31]:
x.size()[1]

3

In [36]:
8lkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk5k548bh45

tensor([[1.9495e+08, 3.0791e-41, 0.0000e+00, 0.0000e+00, 1.4013e-45]])

torch.Size([10])


### Simple MLP

In [20]:
class MLP(nn.Module):
    def __init__(self, num_features, num_hid, num_out):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(num_features, num_hid),
            nn.ReLU(),
            nn.Linear(num_hid, num_out)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [15]:
X_train, X_valid, y_train, y_valid = \
    train_test_split(train_df.iloc[:, 1:], train_df['y'], test_size=3/10, random_state=42)

In [22]:
X_train.shape

(773, 1524)

In [25]:
mlp_model = MLP(1524, 100, 1)
print(mlp_model)

MLP(
  (model): Sequential(
    (0): Linear(in_features=1524, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)


In [34]:
torch.tensor(X_valid.values).size()

torch.Size([332, 1524])